In [ ]:
# Login using username and password

# Import necesary Modules

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium.webdriver.common.by import By
import time

website = 'https://www.linkedin.com/home'
path = "D:/New folder/chromedriver_win32/chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get(website)
email_fiels = driver.find_element(By.ID,'session_key')
email_fiels.send_keys('Email')

password_field = driver.find_element(By.ID,'session_password')
password_field.send_keys('Password')
signin_button = driver.find_element(By.XPATH,'//*[@id="main-content"]/section[1]/div/div/form[1]/div[2]/button')
signin_button.click()

time.sleep(5)

search_button = driver.find_element(By.XPATH,'//*[@id="global-nav-typeahead"]/input')
search_button.send_keys('Data analyst')
search_button.send_keys(Keys.RETURN)
time.sleep(5)

jobs_button = driver.find_element(By.XPATH,'//*[@id="search-reusables__filters-bar"]/ul/li[1]/button')
jobs_button.click() 


In [ ]:
# creating empty list 

job_title = []
job_description = []
company_Name = []
job_Link = []
experience = []

In [ ]:
# after login extrating job_title, job_description, company_Name, job_Link

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
for k in range(1,41):
    page_numbers = driver.find_element(By.XPATH,f'//button[@aria-label="Page {k}"]')
    driver.execute_script('arguments[0].click();',page_numbers)
    
    jobs_block = driver.find_elements(By.XPATH,'//ul[@class="scaffold-layout__list-container"]/li')
    for i in jobs_block:
        driver.execute_script("arguments[0].scrollIntoView();",i)
        time.sleep(1)
        
    job_links = driver.find_elements(By.XPATH,'//ul[@class="scaffold-layout__list-container"]/li/div/div/div[1]/div[2]/div[1]/a')
    
    
    
    for link in job_links:
        
        WebDriverWait(driver, 20).until(EC.element_to_be_clickable(link)).click()
        time.sleep(5)
        job_title.append(driver.find_element(By.CLASS_NAME,'jobs-unified-top-card__job-title').text)
        
        job_description.append(driver.find_element(By.CLASS_NAME,'jobs-description-content').text)
        
        company_Name.append(driver.find_element(By.CLASS_NAME,'jobs-unified-top-card__company-name').text)
        
        
        ab = driver.find_element(By.XPATH,'/html/body/div[5]/div[3]/div[4]/div/div/main/div/section[2]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/a')
        job_Link.append(ab.get_attribute('href'))
        
        
        

    

In [ ]:
# Cleaning the Job description using stop words an stemming

import re
from nltk.corpus import stopwords
z = []
# filter only alphabetic words and exclude like *,@,!,,,,
for i in range(len(job_description)):
    words_AZ = re.sub(pattern = '[^0-9a-zA-Z+]',repl = ' ',string = job_description[i])
    
    #made each word by comma seperated because of stopwords
    words_split = words_AZ.split() # it is used only when give the input to stem function
   
    # Removing stop words like 'and', 'or',,,,
    stop_words = [i for i in words_split if i not in stopwords.words('english')]
    
    # Joining the stemmed words
    z.append(" ".join(stop_words)) # .join is necessary only when we use stemming of words because stem gives each word by comma seperated 



In [ ]:
# By using Regex function we can extract the experience in the job description and store in the job description list

pattern = '[0-9]+ year|[0-9]+[+] year|[0-9]+ [to]{2} [0-9]+ year|[0-9]+\-year|[0-9]+\-[0-9]+ year|[0-9]+ \- [0-9]+ year|[0-9]+ \- year'

for i in z:
    pattern_match =re.findall(pattern,i)
    set_match = set(pattern_match)
    a = ', '.join(set_match)


    if len(a)==0:
        a = 'Freshers or Not specified'
    else:a = a
    experience.append(a)

In [ ]:
# Convert list of columns into dataframe

dict = {'company_Name':company_Name,'job_title':job_title,'experience':experience,'job_description':job_description,'job_Link':job_Link}
df = pd.DataFrame(dict)
df

In [ ]:
# Finally stored into csv file

df.to_csv("D:/New folder/job_dsc2.csv")